В этом ноутбуке я обработал данные вакансий для загрузки в базу данных и дальнейшего моделирования.
Шаг обработки следующие:
- ‌id — проверка и удаление дубликатов

- name — привести к нижнему регистру, убрать стопслова, убрать грейды

- city — убрать записи с лишними городами 

- salary — удалить слишком маленькие значения (=валюта не рубли), заполнить пустые значения from/to значениями to/from, добавление медианного значения (mid_salary)

- employer — удалить строки с ИП и ФИО, НИЦ/НИИ/НТЦ привести к значению "институты", убрать из названий ГК/ООО/АО и т.д.

- work_format — всё, что содержит "гибрид", заменить на гибрид, где remote — удалённо, on_site — в офисе, в остальных случаях, если есть field work, то удалить записи

- experience — преобразовать, добавить фичу: нет опыта — intern/junior, 1-3 — jun+/middle, 3-6 — middle+/senior

- key_skills — заполнить пропуски частотными навыками по группам

# Базовая информация

In [568]:
# Импортируем библиотеки
import numpy as np
import pandas as pd
from collections import Counter
import re

In [569]:
# Загружаем сырые данные и выводим основную информацию
cities = pd.read_excel(r'Z:\Repos\mlops2025\hr_analytics\data\raw\cities.xlsx')
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2191 non-null   int64  
 1   name         2191 non-null   object 
 2   group        2191 non-null   object 
 3   city         2191 non-null   object 
 4   salary_from  1690 non-null   float64
 5   salary_to    1464 non-null   float64
 6   employer     2191 non-null   object 
 7   work_format  2191 non-null   object 
 8   experience   2191 non-null   object 
 9   key_skills   2191 non-null   object 
 10  description  2191 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 188.4+ KB


In [570]:
cities.shape

(2191, 11)

In [571]:
# Посмотрим на первые 5 строк
cities.head()

,id,name,group,city,salary_from,salary_to,employer,work_format,experience,key_skills,description
0,126298182,Системный/бизнес аналитик (стажер),Системный аналитик,Казань,NaN,23200.0,ICL Services,['REMOTE'],Нет опыта,"['Аналитическое мышление', 'Системный анализ',...",Мы – российская продуктово-сервисная ИТ-компан...
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,NaN,ТАТТЕЛЕКОМ,['ON_SITE'],От 1 года до 3 лет,[],ПАО «Таттелеком» — один из ключевых операторов...
2,126319001,QA-инженер (senior),Тестировщик,Казань,150000.0,170000.0,ГК Innostage,['REMOTE'],От 3 до 6 лет,"['REST', 'SOAP', 'SQL', 'Git', 'Ручное тестиро...","Компания Innostage (Инностейдж) - IT компания,..."
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,Кинофлекс,['REMOTE'],От 1 года до 3 лет,"['Figma', 'UI', 'UX']",Компания FLEX — это инновационный онлайн-кинот...
4,126243205,Junior Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,Первый Бит,[],От 1 года до 3 лет,"['Python', 'Docker', 'SQL', 'FastAPI']",Обязанности: Поддержка и развитие чат-бота в ...


# Обработка id

In [572]:
# Проверим, нет ли повторяющихся вакансий
# Если они есть, удалим их
df_duplicates = cities[cities.duplicated(subset=['id'], keep=False)].sort_values(by='id')
df_duplicates.shape  # 50 записей - дубликаты

(50, 11)

In [573]:
# Удалим дубликаты
vacancies = cities.copy()  # работаем с копией данных
vacancies.drop_duplicates(subset='id', keep='first', inplace=True)
vacancies.shape

(2166, 11)

# Обработка городов

In [574]:
# Проверим, нет ли лишних городов
vacancies['city'].unique()

array(['Казань', 'Москва', 'Санкт-Петербург', 'Уфа', 'Екатеринбург',
       'Краснодар', 'Новосибирск', 'Пермь', 'Тюмень', 'Ноябрьск',
       'Челябинск', 'Первоуральск', 'Ревда (Свердловская область)',
       'Академгородок (Новосибирская область)', 'Бердск', 'Сочи',
       'Ломоносов', 'Белоостров', 'Колпино', 'Обнинск', 'Зеленоград',
       'Жуковский  (Московская область)', 'Томск', 'Балаково', 'Оренбург',
       'Дмитров (Московская область)', 'Лобня (Московская область)',
       'Самара', 'Курск', 'Воронеж', 'Тверь'], dtype=object)

In [575]:
# Удалим записи с лишними городами
city_list = ['Москва', 'Санкт-Петербург', 'Казань', 'Новосибирск', 'Екатеринбург']
vacancies = vacancies[vacancies['city'].isin(city_list)].reset_index(drop=True)

# Проверка городов
vacancies['city'].unique()


array(['Казань', 'Москва', 'Санкт-Петербург', 'Екатеринбург',
       'Новосибирск'], dtype=object)

# Обработка зарплат

In [576]:
# Заполним пустые значения from и to значениями to и from

# Заполняем нижнюю границу верхней, если поле пустое
vacancies['salary_from'] = vacancies['salary_from'].fillna(vacancies['salary_to'])

# То же самое с верхней границей
vacancies['salary_to'] = vacancies['salary_to'].fillna(vacancies['salary_from'])

# Заполняем медианой новый столбец: средним списка из двух
vacancies['mid_salary'] = np.median([vacancies['salary_from'], vacancies['salary_to']], axis=0)

# Перегруппируем столбцы для лучшего отображения
cols = vacancies.columns.to_list()
vac = vacancies[['id',
           'name',
           'group',
           'city',
           'salary_from',
           'salary_to',
           'mid_salary',
           'employer',
           'work_format',
           'experience',
           'key_skills',
           'description',
           ]]

vac.head()

,id,name,group,city,salary_from,salary_to,mid_salary,employer,work_format,experience,key_skills,description
0,126298182,Системный/бизнес аналитик (стажер),Системный аналитик,Казань,23200.0,23200.0,23200.0,ICL Services,['REMOTE'],Нет опыта,"['Аналитическое мышление', 'Системный анализ',...",Мы – российская продуктово-сервисная ИТ-компан...
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,175500.0,175500.0,ТАТТЕЛЕКОМ,['ON_SITE'],От 1 года до 3 лет,[],ПАО «Таттелеком» — один из ключевых операторов...
2,126319001,QA-инженер (senior),Тестировщик,Казань,150000.0,170000.0,160000.0,ГК Innostage,['REMOTE'],От 3 до 6 лет,"['REST', 'SOAP', 'SQL', 'Git', 'Ручное тестиро...","Компания Innostage (Инностейдж) - IT компания,..."
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,145000.0,Кинофлекс,['REMOTE'],От 1 года до 3 лет,"['Figma', 'UI', 'UX']",Компания FLEX — это инновационный онлайн-кинот...
4,126243205,Junior Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,70000.0,Первый Бит,[],От 1 года до 3 лет,"['Python', 'Docker', 'SQL', 'FastAPI']",Обязанности: Поддержка и развитие чат-бота в ...


In [577]:
vac.shape

(2129, 12)

In [578]:
# Удалим значения со слишком малыми значениями
vac = vac[vac['mid_salary'] > 10000]
vac.shape

(2077, 12)

# Обработка работодателя

In [579]:
pd.set_option('display.max_rows', None)

In [581]:
# Обработаем столбец с работодателями

vac['employer'] = vac['employer'].str.lower()

# Унифицируем научные предприятия
pat_science = r"НПЦ|НИИ|НТЦ|НПО|научно-производственное объединение|НПК|Научно-Исследовательский Институт|ВНИИ|НПП|НИЦ|лаборатория|научно-производственное предприятие|информационный центр|нтпц".lower()
vac['employer'] = vac['employer'].where(~vac['employer'].str.contains(pat_science, case=False, na=False), "научные предприятия")

# Унифицируем государственные предприятия
pat_state = r"ФКУ|ФГАНУ|ФГУП|ФГБУ|Городской информационный центр|Республики|Филиал|гку|оцрв|ано|фбу|гбу|офд".lower()
vac['employer'] = vac['employer'].where(~vac['employer'].str.contains(pat_state, case=False, na=False), "госкорпорация")
vac['employer'] = vac['employer'].str.replace(r"\([^)]*\)", "", regex=True)  # удаляем скобки, и всё, что внутри
vac['employer'] = vac['employer'].str.replace(r",.*$", "", regex=True)  # удаляем всё, что идёт после запятой

# Обрабатываем сбер и газпром
vac['employer'] = vac['employer'].where(~vac['employer'].str.contains('газпром'), 'газпром')
vac['employer'] = vac['employer'].where(~vac['employer'].str.contains('сбер') & ~vac['employer'].str.contains('sber'), 'сбер')

# Обрабатываем значения после специальных символов: / и |
vac['employer'] = vac['employer'].str.replace(r"/.*$", "", regex=True)
vac['employer'] = vac['employer'].str.replace(r"\|.*$", "", regex=True)

# Обработаем ещё несколько неинформативных частей названий
vac['employer'] = vac['employer'].str.replace("llc", "")
vac['employer'] = vac['employer'].str.replace("ltd.", "")
vac['employer'] = vac['employer'].str.replace("pte.", "")
vac['employer'] = vac['employer'].str.replace("ао ", "")
vac['employer'] = vac['employer'].str.replace("гк", "")
vac['employer'] = vac['employer'].str.replace("llc", "")
vac = vac.drop(index=[391, 746, 1307, 1455, 1491])

# Обрабатываем вакансии, где работодатель - фио, а не компания
name_list = [
    'мухин дмитрий сергеевич',
    'чепурин алексей владимирович',
    'дрынкин андрей александрович',
    'сизинцев роман александрович',
    'колпаков денис викторович',
    'лебедев роман игоревич',
    'токарева александра маратовна',
    'буров алексей константинович',
    'харламов денис сергеевич',
    'бабаян денис маркович',
    'шиховец александр викторович',
    'росляков никита александрович',
    'коваленко юрий николаевич',
    'мамедов эльдар юнисович',
    'бордаковский антон робертович',
]
vac = vac[~vac['employer'].isin(name_list)]

# В конце на всякий случай чистим лишние пробелы
vac['employer'] = vac['employer'].str.strip()

In [583]:
vac.shape

(2056, 12)

# Обработка рабочего формата

In [584]:
import ast  # приведём навыки к списку

def work_format_type(text):
    try:
        return ast.literal_eval(text)
    except (ValueError, SyntaxError):
        return []

vac['work_format'] = vac['work_format'].apply(work_format_type)
vac['work_format'].iloc[0]

['REMOTE']

In [585]:
vac['work_format'].value_counts()

work_format
[REMOTE]                                 717
[ON_SITE]                                512
[]                                       203
[ON_SITE, HYBRID]                        177
[ON_SITE, REMOTE, HYBRID]                150
[HYBRID]                                 125
[REMOTE, HYBRID]                          89
[FIELD_WORK]                              22
[ON_SITE, REMOTE]                         21
[ON_SITE, FIELD_WORK]                     21
[ON_SITE, HYBRID, FIELD_WORK]              7
[HYBRID, FIELD_WORK]                       6
[ON_SITE, REMOTE, HYBRID, FIELD_WORK]      3
[REMOTE, FIELD_WORK]                       2
[REMOTE, HYBRID, FIELD_WORK]               1
Name: count, dtype: int64

In [586]:
vac.head()

,id,name,group,city,salary_from,salary_to,mid_salary,employer,work_format,experience,key_skills,description
0,126298182,Системный/бизнес аналитик (стажер),Системный аналитик,Казань,23200.0,23200.0,23200.0,icl services,[REMOTE],Нет опыта,"['Аналитическое мышление', 'Системный анализ',...",Мы – российская продуктово-сервисная ИТ-компан...
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,175500.0,175500.0,таттелеком,[ON_SITE],От 1 года до 3 лет,[],ПАО «Таттелеком» — один из ключевых операторов...
2,126319001,QA-инженер (senior),Тестировщик,Казань,150000.0,170000.0,160000.0,innostage,[REMOTE],От 3 до 6 лет,"['REST', 'SOAP', 'SQL', 'Git', 'Ручное тестиро...","Компания Innostage (Инностейдж) - IT компания,..."
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,145000.0,кинофлекс,[REMOTE],От 1 года до 3 лет,"['Figma', 'UI', 'UX']",Компания FLEX — это инновационный онлайн-кинот...
4,126243205,Junior Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,70000.0,первый бит,[],От 1 года до 3 лет,"['Python', 'Docker', 'SQL', 'FastAPI']",Обязанности: Поддержка и развитие чат-бота в ...


In [587]:
def neat_work_format(lst):
    if not lst:  # пустой список
        return 'не указано'
    if len(lst) == 1:
        if "REMOTE" in lst:
            return 'удалённо'
        elif "ON_SITE" in lst:
            return 'в офисе'
        elif "FIELD_WORK" in lst:
            return 'выездная работа'
    if 'HYBRID' in lst and 'FIELD_WORK' not in lst:
        return 'гибрид'
    if 'FIELD_WORK' in lst:
        return 'выездная работа'
    return 'не указано'


In [588]:
vac['work_format'] = vac['work_format'].apply(neat_work_format)
vac.head()

,id,name,group,city,salary_from,salary_to,mid_salary,employer,work_format,experience,key_skills,description
0,126298182,Системный/бизнес аналитик (стажер),Системный аналитик,Казань,23200.0,23200.0,23200.0,icl services,удалённо,Нет опыта,"['Аналитическое мышление', 'Системный анализ',...",Мы – российская продуктово-сервисная ИТ-компан...
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,175500.0,175500.0,таттелеком,в офисе,От 1 года до 3 лет,[],ПАО «Таттелеком» — один из ключевых операторов...
2,126319001,QA-инженер (senior),Тестировщик,Казань,150000.0,170000.0,160000.0,innostage,удалённо,От 3 до 6 лет,"['REST', 'SOAP', 'SQL', 'Git', 'Ручное тестиро...","Компания Innostage (Инностейдж) - IT компания,..."
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,145000.0,кинофлекс,удалённо,От 1 года до 3 лет,"['Figma', 'UI', 'UX']",Компания FLEX — это инновационный онлайн-кинот...
4,126243205,Junior Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,70000.0,первый бит,не указано,От 1 года до 3 лет,"['Python', 'Docker', 'SQL', 'FastAPI']",Обязанности: Поддержка и развитие чат-бота в ...


In [589]:
vac['work_format'].value_counts()

work_format
удалённо           717
гибрид             541
в офисе            512
не указано         224
выездная работа     62
Name: count, dtype: int64

In [590]:
vac = vac[vac['work_format'] != 'выездная работа']

In [591]:
vac.shape

(1994, 12)

# Обработка опыта

In [592]:
vac.head()

,id,name,group,city,salary_from,salary_to,mid_salary,employer,work_format,experience,key_skills,description
0,126298182,Системный/бизнес аналитик (стажер),Системный аналитик,Казань,23200.0,23200.0,23200.0,icl services,удалённо,Нет опыта,"['Аналитическое мышление', 'Системный анализ',...",Мы – российская продуктово-сервисная ИТ-компан...
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,175500.0,175500.0,таттелеком,в офисе,От 1 года до 3 лет,[],ПАО «Таттелеком» — один из ключевых операторов...
2,126319001,QA-инженер (senior),Тестировщик,Казань,150000.0,170000.0,160000.0,innostage,удалённо,От 3 до 6 лет,"['REST', 'SOAP', 'SQL', 'Git', 'Ручное тестиро...","Компания Innostage (Инностейдж) - IT компания,..."
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,145000.0,кинофлекс,удалённо,От 1 года до 3 лет,"['Figma', 'UI', 'UX']",Компания FLEX — это инновационный онлайн-кинот...
4,126243205,Junior Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,70000.0,первый бит,не указано,От 1 года до 3 лет,"['Python', 'Docker', 'SQL', 'FastAPI']",Обязанности: Поддержка и развитие чат-бота в ...


In [593]:
vac['experience'] = vac['experience'].str.lower()
vac.head()

,id,name,group,city,salary_from,salary_to,mid_salary,employer,work_format,experience,key_skills,description
0,126298182,Системный/бизнес аналитик (стажер),Системный аналитик,Казань,23200.0,23200.0,23200.0,icl services,удалённо,нет опыта,"['Аналитическое мышление', 'Системный анализ',...",Мы – российская продуктово-сервисная ИТ-компан...
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,175500.0,175500.0,таттелеком,в офисе,от 1 года до 3 лет,[],ПАО «Таттелеком» — один из ключевых операторов...
2,126319001,QA-инженер (senior),Тестировщик,Казань,150000.0,170000.0,160000.0,innostage,удалённо,от 3 до 6 лет,"['REST', 'SOAP', 'SQL', 'Git', 'Ручное тестиро...","Компания Innostage (Инностейдж) - IT компания,..."
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,145000.0,кинофлекс,удалённо,от 1 года до 3 лет,"['Figma', 'UI', 'UX']",Компания FLEX — это инновационный онлайн-кинот...
4,126243205,Junior Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,70000.0,первый бит,не указано,от 1 года до 3 лет,"['Python', 'Docker', 'SQL', 'FastAPI']",Обязанности: Поддержка и развитие чат-бота в ...


In [594]:
def grade_col(text):
    text = text.lower().strip()
    if text == 'нет опыта':
        return 'стажёр/junior'
    elif text == 'от 1 года до 3 лет':
        return 'junior+/middle'
    else:
        return 'middle+/senior'

vac['grade'] = vac['experience'].apply(grade_col)


# Обработка ключевых навыков

In [595]:
import ast  # приведём навыки к списку

def parse_skills(text):
    try:
        return ast.literal_eval(text)
    except (ValueError, SyntaxError):
        return []

vac['key_skills'] = vac['key_skills'].apply(parse_skills)
vac['key_skills'].iloc[0]

['Аналитическое мышление', 'Системный анализ', 'Бизнес-анализ']

In [596]:
# Теперь необходимо определить самые частотные навыки по роли и заполнить ими пустые списки
vac['key_skills'].iloc[6]

[]

In [597]:
# Будем брать топ-5 навыков
top_skills_group = {}

for group, group_df in vac.groupby('group'):
    all_skills = sum(group_df['key_skills'].to_list(), [])
    # Создаём объект счётчика
    skill_counts = Counter(all_skills)
    # CСобираем топ-скиллы
    top_skills = [skill for skill, count in skill_counts.most_common(5)]
    top_skills_group[group] = top_skills

# Напишем функцию для заполнения пустых списков навыков по ролям
def fill_empty_skills(row):
    if isinstance(row['key_skills'], list) and len(row['key_skills'])== 0:
        # Заполняем значениями и меняем дефолт с None на пустой список
        return top_skills_group.get(row['group'], [])
    else:
        return row['key_skills']

# Применяем функцию
vac['key_skills'] = vac.apply(fill_empty_skills, axis=1)

In [598]:
# По ходу обработки я понял, что столбец описания не понадобится, так что удалим его
vac.drop(columns=['description'], inplace=True)

# Обработка названий

In [599]:
# Удалим грейды и всё, что в скобках

grades = [
    'intern',
    'junior',
    'juniormiddle',
    'lead',
    'middle',
    'middle senior',
    'senior',
    'teamlead',
    'trainee',
    'ведущий',
    'главный',
    'младший',
    'начинающий',
    'стажер',
    'стажёр',
    'тимлид',
]

def clean_grades_and_parentheses(series, grades_list):
    pattern_grades = r'\b(?:' + '|'.join(re.escape(g) for g in grades_list) + r')\b'
    pattern_parentheses = r'\([^)]*\)'

    def clean_text(text):
        if not isinstance(text, str):
            return text
        # Удаляем скобки и содержимое
        text = re.sub(pattern_parentheses, '', text)
        # Удаляем грейды
        text = re.sub(pattern_grades, '', text, flags=re.IGNORECASE)
        # Убираем лишние пробелы
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    return series.apply(clean_text)

vac['name'] = clean_grades_and_parentheses(vac['name'], grades)


# Сохранение очищенных данных

In [602]:
vac.head()

,id,name,group,city,salary_from,salary_to,mid_salary,employer,work_format,experience,key_skills,grade
0,126298182,Системный/бизнес аналитик,Системный аналитик,Казань,23200.0,23200.0,23200.0,icl services,удалённо,нет опыта,"[Аналитическое мышление, Системный анализ, Биз...",стажёр/junior
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,175500.0,175500.0,таттелеком,в офисе,от 1 года до 3 лет,"[Git, PostgreSQL, JavaScript, SQL, Linux]",junior+/middle
2,126319001,QA-инженер,Тестировщик,Казань,150000.0,170000.0,160000.0,innostage,удалённо,от 3 до 6 лет,"[REST, SOAP, SQL, Git, Ручное тестирование, Ag...",middle+/senior
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,145000.0,кинофлекс,удалённо,от 1 года до 3 лет,"[Figma, UI, UX]",junior+/middle
4,126243205,Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,70000.0,первый бит,не указано,от 1 года до 3 лет,"[Python, Docker, SQL, FastAPI]",junior+/middle


In [601]:
vac.shape

(1994, 12)

In [603]:
vac.to_excel('Z:\Repos\mlops2025\hr_analytics\data\processed\processed.xlsx', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
C:\Users\Artemis\AppData\Local\Temp\ipykernel_14348\1414643131.py:1: SyntaxWarning: invalid escape sequence '\R'
  vac.to_excel('Z:\Repos\mlops2025\hr_analytics\data\processed\processed.xlsx', index=False)
